In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
df_last_15_days = df.loc[df['timestamp'] > pd.to_datetime('2018-05-13')]

df_15_previous_days = df.loc[df['timestamp'] > pd.to_datetime('2018-05-01')]
df_15_previous_days = df_15_previous_days.loc[df_15_previous_days['timestamp'] < pd.to_datetime('2018-05-13')]

### Vamos a darle el formato necesario al primer df

In [4]:
event_count_last_15 = df_last_15_days.groupby(['person', 'event']).agg({'timestamp':'count'})

In [5]:
event_count_prev_15 = df_15_previous_days.groupby(['person', 'event']).agg({'timestamp':'count'})

In [6]:
event_count_last_15 = event_count_last_15.reset_index()

In [7]:
event_count_last_15.columns = ['person', 'event', 'count']
event_count_last_15 = event_count_last_15.pivot_table(index='person', columns='event', values='count')

In [8]:
event_count_last_15 = event_count_last_15.fillna(0)

In [11]:
event_count_last_15.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
00091926,12.0,24.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0,23.0
000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,339.0,13.0


### Ahora vamos a darle el formato al otro df

In [13]:
event_count_prev_15.reset_index(inplace=True)

In [16]:
event_count_prev_15.columns = ['person', 'event', 'count']
event_count_prev_15 = event_count_prev_15.pivot_table(index='person', columns='event', values='count')

In [19]:
event_count_prev_15 = event_count_prev_15.fillna(0)

In [20]:
event_count_prev_15.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
00091926,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,11.0
0019e639,2.0,15.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,13.0,4.0
001ca5ee,3.0,2.0,0.0,0.0,6.0,0.0,3.0,0.0,0.0,7.0,2.0
001e9aea,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0020152e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


### Ahora unimos ambos dfs

In [46]:
import math

result = pd.DataFrame()
for column in event_count_last_15.columns:
    result[column] = (event_count_last_15[column] - event_count_prev_15[column])

In [47]:
result = result.fillna(0)

In [48]:
result = result[['checkout', 'visited site', 'searched products', 'viewed product']]
result.columns = [
    'diff_checkouts_15_dias',
    'diff_visited_site_15_dias',
    'diff_searched_products_15_dias',
    'diff_viewed_products_15_dias',
]

In [49]:
result.to_csv('../data/diferencia_de_count_de_eventos.csv')